# Продвинутый pandas 2

# Задание 1.

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [4]:
import pandas as pd
data = pd.read_csv('ratings.csv')

df1 = data.groupby('userId').timestamp.max() - data.groupby('userId').timestamp.min() #время "жизни" каждого пользователя
df2 = data.groupby('userId').count().rating #подсчет суммарного количества оценок каждого пользователя

df3 = df1.to_frame().join(df2).reset_index() #объединение двух series
df3.columns = ['userId', 'av_timestamp', 'count']    #изменение названия столбцов

print('Среднее время "жизни" пользователей, выставивших более 100 оценок:', 
      round(df3.loc[df3['count']>=100, :].av_timestamp.mean()/3600, 1), 'часов.')


Среднее время "жизни" пользователей, выставивших более 100 оценок: 10925.5 часов.


# Задание 2.

Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки;
- auto - автомобильные перевозки;
- air - воздушные перевозки;
- client_base - адреса клиентов.

Необходимо сформировать две таблицы:

 1) таблицу с тремя типами выручки для каждого client_id без указания адреса клиента;    
 2) аналогичную таблицу по типам выручки с указанием адреса клиента.

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [5]:
import pandas as pd
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [9]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [6]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [7]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [10]:
#суммируем выручку клиентов с одинаковым id (на случай, если будут)
rzd = rzd.groupby('client_id').sum().reset_index()
auto = auto.groupby('client_id').sum().reset_index()
air = air.groupby('client_id').sum().reset_index()

#Таблица 1. Три типа выручки для каждого client_id без указания адреса клиента.

rzd.merge(auto, how='outer').merge(air, how='outer').fillna(0)


,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [11]:
# Таблица 2. Три типа выручки для каждого client_id с указанием адреса клиента.

rzd.merge(auto, how='outer').merge(air, how='outer').merge(client_base, how='outer').fillna(0)


,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,0.0,0.0,Комсомольская 4
1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,113,10283.0,57483.0,0.0,Левобережная 1а
3,114,5774.0,83.0,0.0,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,0.0,4834.0,4.0,Строителей 18
6,117,0.0,98.0,13.0,Панфиловская 33
7,118,0.0,0.0,173.0,Мастеркова 4


# Задание 3.

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. 
Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению 
для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? 
Для составления ответа можно использовать вопросы:
1. У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
2. Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
3. Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.


Ответы:

Вначале можно воспользоваться регулярными выражениями и записать в отдельных столбцах таблицы с местоположением широту и долготу по шаблону. Затем объединить две таблицы по столбцу timestamp, который должен присутствовать в обеих таблицах. Без времени (только по id пользователя) не получится корректно объединить таблицы. Для объединения можно использовать метод merge() с дополнительным параметром outer, чтобы сохранить все данные из двух таблиц.

Если наборы координат одного и того же пользователя значительно удалены друг от друга, то это никак не должно повлиять на применение метода merge(), описанного выше. Удаленность координат говорит о том, что пользователь в моменты своих визитов/покупок на сайте в разное время находился в разных местах. 

По координатам можно определить такой признак пользователя, как его местоположение: страна, город, район и т.п. Если позволяет частота визитов пользователя, то можно предположить место его жительства, работы, отдыха, сопоставив время визитов и координаты. А имея такие данные, можно пользователю отправлять целевую рекламу (например, акции в магазинах рядом с его домом; акции в кафе рядом с его работой...). 

Также можно поискать закономерность между местонахождением пользователя (на работе, дома, в отпуске), конверсией и видом покупки. Определить активность передвижений пользователя.

Таким образом, зная координаты пользователя в момент визитов и покупок, можно вывести множество различных производных для более глубого анализа картины пользователя. 